In [55]:
# Generate video for a program. Make sure you have the executable open
import sys
import os
import re
import json
import glob
import csv
import copy
import time
import random
import threading
import IPython.display
from pathlib import Path
from PIL import Image

sys.path.append('../simulation/')
sys.path.append('../dataset_utils/')

import add_preconds
import augmentation_utils
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from utils_demo import *
from unity_simulator.comm_unity import UnityCommunication
from unity_simulator import utils_viz

equiv_class = utils.load_name_equivalence()
prefab_class = utils.load_name_equivalence(file_name='../../resources/PrefabClass.json')

rooms = ["bedroom", "livingroom", "kitchen", "bathroom"]

alternative_temp_action = {
    "PUSH" : "TOUCH",
    "WASH" : "RINSE",
    "WAKEUP" : "STANDUP",
#     "LIE" : "LAYDOWN",## アニメーションのせいで
    "PLUGIN" : "FIND",## 代えられるアクション無し
    "PULL" : "FIND",## 代えられるアクション無し
    "PUTON" : "PUT",## 代えられるアクション無し
    "PLUGOUT" : "FIND",## 代えられるアクション無し
}

# class_name_equivalenceにあるかもしれませんが
# graphに実際存在している名前
alternative_temp_obj = {
    "remote_control" : "remotecontrol",
    "washing_machine" : "washingmachine",
    "clothes_pile" : "clothespile",# basket_for_clothes
    "pants" : "clothespants",# clothes_pants
    "shirt" : "clothesshirt",# clothes_shirt
    "electrical_outlet" : "powersocket",
    "kitchen_cabinets" : "kitchencabinet",# cupboard
    "cutlery_fork" : "cutleryfork",# fork
    "cutlery_knife" : "cutleryknife",# knife
    "kitchen_table" : "kitchentable",
    "coffee_table" : "coffeetable",
    "diningtable" : "kitchentable",# table
    "wall_phone" : "wallphone",
    "lamp" : "tablelamp",# floor_lamp
    "dish_bowl" : "dishbowl",# bowl
    "bottle_water" : "bottlewater",# water
    "kitchen_counter" : "kitchencounter",
    "lotionbottle" : "hairproduct",# shampoo
    "washing_sponge" : "washingsponge",# sponge
    "picture" : "wallpictureframe",# picture
    "rag" : "towel",
    "bread" : "breadslice",# food_bread
    "garbage_can" : "garbagecan",
    "papertowel" : "napkin",# paper_towel
}

# id_mappingが必要ないアクション
no_id_mapping_actions = ['SWEEP', 'STIR', 'FALLSIT']

change_execute_map = {'lightswitch':'light', 'tv':'television', 'remotecontrol':'controller', 'sofa':'couch'}
room_map = {'home_office':'livingroom', 'living_room':'livingroom', 'entrance_hall':'livingroom',
            'kitchen':'kitchen', 'dining_room':'kitchen',
            'bathroom':'bathroom', 
            'bedroom':'bedroom', 'kids_bedroom':'bedroom'}

fixed_cameras = {
    'kitchen':{1:[63,64,65,66], 2:[39,40,41,42], 3:[28,29,30,31], 4:[30,31,32,33], 5:[49,50,51,52], 6:[61,62,63,64], 7:[41,42,43,44]},
    'bathroom':{1:[13,14,15,16], 2:[9,10,11,12], 3:[79,80,81,82], 4:[45,46,47,48], 5:[80,81,82,83], 6:[12,13,14,15], 7:[11,12,13,14]},
    'bedroom':{1:[38,39,40,41], 2:[60,61,62,63], 3:[61,62,63,64], 4:[65,66,67,68], 5:[64,65,66,67], 6:[34,35,36,37], 7:[89,90,91,92]},
    'livingroom':{1:[86,87,88,89], 2:[82,83,84,85], 3:[47,48,49,50], 4:[91,92,93,94], 5:[16,17,18,19], 6:[84,85,86,87], 7:[67,68,69,70]}
}

In [56]:
def get_program_file(file_path):
    program_name = ''
    description = ''
    list_of_steps = []
    with open(file_path, "r") as f:
        lines = f.readlines()
        program_name = lines[0].replace("\n", "")
        description = lines[1].replace("\n", "")
        for line in lines[2:]:
            line = line.replace("\n", "")
            if line.startswith("["):
                list_of_steps.append(line)
            else:
                pass

    return program_name, description, list_of_steps

def switchOff(g):
    off_list = ['lightswitch', 'light', 'tv', 'television', 'computer', 'faucet']
    for class_name in off_list:
        nodes = find_nodes(g, class_name=class_name)
        for node in nodes:
            states = [x for x in node['states'] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            node['states'] = states
    return g

def update_states(pre_graph, graph_state_list, obj_id_map):#, modified_graph
    if (len(obj_id_map) > 0):
        try:
            new_graph = copy.deepcopy(pre_graph)
            for new_node in new_graph['nodes']:
#                 if obj_id_map is not None and modified_graph is not None:
#                     for k,v in obj_id_map.items():
#                         if new_node['id'] == v:
#                             new_node['states'] = [node['states'] for node in modified_graph['nodes'] if node['id'] == v]
#                             new_node['properties'] = [node['properties'] for node in modified_graph['nodes'] if node['id'] == v]
                for k,v in obj_id_map.items():
                    if new_node['id'] == v:
                        for node in graph_state_list['nodes']:
                            if node['id'] == v:
                                new_node['states'] = node['states']
                                new_node['properties'] = node['properties']
#             if obj_id_map is not None and modified_graph is not None:
#                 comm.expand_scene(new_graph)
            return new_graph
        except Exception as e:
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(e)

def export(graph_state_list, script_lines, activity_name, description, durations):
    file_path = "../simulation/Output/scene"+str(scene_id)+"/"+tsv_data['file_name'][:-4]+"/graph states/"
    if os.path.exists(file_path) == False:
        os.mkdir(file_path)
    file_index = 0
    for graph in graph_state_list:
        file_index += 1
        json_path = file_path + "activityList-graph-state-" + '{0:03d}'.format(file_index) + ".json"
        with open(json_path, 'w') as outfile:
            json.dump(graph, outfile, indent=2)
        json_path = ''

    script_path = file_path + "activityList-program.txt"
    with open(script_path, 'w') as f:
        for s in script_lines:
            f.write("%s\n" % s)

    description_path = file_path + "program-description.txt"
    with open(description_path, 'w') as f:
        f.write("%s\n" % activity_name)
        f.write("%s\n" % description)

    duration_path = file_path + "duration.txt"
    time_list = [str(time) for time in durations]
    duration = "\n".join(time_list)
    with open(duration_path, 'w') as f:
        f.write(duration)

    program_path = file_path + "executed-program.txt"
    with open(program_path, 'w') as f:
        f.write("%s\n" % activity_name)
        f.write("%s\n\n\n" % description)
        for s in script_lines:
            f.write("%s\n" % s)

def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [57]:
# アクションが二つしかないと思うから
def replace_index(script_line, replace_value1, replace_value2=None):
    start_indexes = [i.start() for i in re.finditer('\(', script_line)]
    end_indexes = [i.start() for i in re.finditer('\)', script_line)]
    if replace_value2 is not None and len(start_indexes) > 1:
        script_line = script_line[:start_indexes[1]+1] + replace_value2 + script_line[end_indexes[1]:]
    script_line = script_line[:start_indexes[0]+1] + replace_value1 + script_line[end_indexes[0]:]
    return script_line

# graphにない場合
def replace_obj_and_get_index(script_line, current_obj):
    # class_name_equivalenceにない場合
    if (equiv_class.get(current_obj) is None):
        script_line, obj_index = get_obj_index_from_alternative(script_line, current_obj)
    else:
        graph_class_names = [node['class_name'] for node in graph['nodes']]
        # find in graph for equaivalence objs
        for equ_new_obj in equiv_class[current_obj]:
            if alternative_temp_obj.get(equ_new_obj) is not None:
                equ_new_obj = alternative_temp_obj[equ_new_obj]
            if equ_new_obj in graph_class_names:
                new_obj = equ_new_obj
                break
            else:
                new_obj = equiv_class[current_obj][0]

        if (alternative_temp_obj.get(new_obj) is None):
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(current_obj+'がgraphに見つからないです。equivalence_classから'+new_obj+'と代わりました。')
            script_line = script_line.replace(current_obj, new_obj)
            action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
            obj_index = get_obj_index(new_obj, action)
            if (obj_index == '1'):
                tsv_data['script'].append(script_line)
                tsv_data['errors'].append(new_obj+'がsceneにいないですから、追加しました。')
                add_node_and_relation(new_obj)
                obj_index = get_obj_index(new_obj, action)
        else:
            # equivalenceの値がalternativeにある場合
            script_line, obj_index = get_obj_index_from_alternative(script_line, new_obj)
    return script_line, obj_index

def get_obj_index_from_alternative(script_line, current_obj):
    action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
    if (alternative_temp_obj.get(current_obj) is not None):
        tsv_data['script'].append(script_line)
        tsv_data['errors'].append(current_obj+'がequivalence_classに見つからないですから'+alternative_temp_obj.get(current_obj)+'と代わりました。')
        new_obj = alternative_temp_obj[current_obj]
        script_line = script_line.replace(current_obj, new_obj)
        obj_index = get_obj_index(new_obj, action)
        if (obj_index == '1'):
            add_node_and_relation(new_obj)
            obj_index = get_obj_index(new_obj, action)
    else:
        if (current_obj in (name['className'].lower() for name in prefab_class['prefabClasses'])):
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(current_obj+'がsceneにいないですから、追加しました。')
            add_node_and_relation(current_obj)
            obj_index = get_obj_index(current_obj, action)
        else:
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(current_obj+'がprefabとして保存していないです。')
            script_line = None
            obj_index = '1'
    return script_line, obj_index

def get_obj_index(obj, action):
    id = [node['id'] for node in graph['nodes'] if node['class_name'] == obj]
    edges = graph['edges']
    if len(id) > 0:
        global current_room
        global current_camera_room
        if obj in rooms:
            current_room = obj
            current_camera_room = obj
            return str(id[0])
        else:
            current_room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == current_room][0]
            ids_in_room = [edge['from_id'] for edge in edges if edge['relation_type'] == 'INSIDE' and edge['to_id'] == current_room_id]
            obj_ids_in_room = [x for x in id if x in ids_in_room]
            if len(obj_ids_in_room) > 0:
                return str(obj_ids_in_room[0])
            else:
                if action in ['WALK', 'FIND', 'PUTOBJBACK']:
                    rooms_object_exist = [edge['to_id'] for edge in edges if edge['relation_type'] == 'INSIDE' and edge['from_id'] == id[0]]
                    room_name_object_exist = [node['class_name'] for node in graph['nodes'] if node['id'] in rooms_object_exist and node['class_name'] in rooms][0]
                    current_camera_room = room_name_object_exist
                return str(id[0])
    else:
        return '1'


def add_object_out_of_script(obj, room, place, relation_type):
    max_id = max(x['id'] for x in graph['nodes']) + 1
    place_id = [node['id'] for node in graph['nodes'] if node['class_name'] == place][0]
    room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == room][0]
    add_node(graph, {'class_name': obj, 
                        'category': '', 
                        'id': max_id, 
                        'properties': [], 
                        'states': []})
    add_edge(graph, max_id, 'INSIDE', room_id)
    add_edge(graph, max_id, relation_type, place_id)
    print(obj, 'is added to the scene on the ', place, 'inside ', room)
    comm.expand_scene(graph)

def add_node_and_relation(obj):
    if (obj not in (x['class_name'] for x in graph['nodes'])):
        add_object_out_of_script(obj, 'kitchen', 'kitchentable', 'ON')

# アクションをチェックし、修正する
def correct_action(script_line):
    action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
    if (alternative_temp_action.get(action) is not None):
        tsv_data['script'].append(script_line)
        tsv_data['errors'].append(action+'というアクションがまだ開発していないですから、'+alternative_temp_action[action]+'と代わりました。')
        return script_line.replace(action, alternative_temp_action[action])
    else:
        if (action == 'GRAB' and objs[0] == 'phone'):
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(action+'はphoneに使うとアクションが続けないで止まりました。FINDに代わってテストしました。')
            return script_line.replace(action, 'FIND')
        if (action == 'DROP' and objs[0] == 'phone'):
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(action+'はphoneに使えませんでした。FINDに代わってテストしました。')
            return script_line.replace(action, 'FIND')
        elif action == 'LIE':
            tsv_data['script'].append(script_line)
            tsv_data['errors'].append(action+'アクションがアニメーションから出てないです。FINDに代わってテストしました。')
            return script_line.replace(action, 'FIND')
        return script_line

In [58]:
def correct_script(script_list):
    global custom_cameras
    global current_camera_room
    result_list = []
    for script_line in script_list:
        script_line = correct_action(script_line)
        action, objs, indexs = augmentation_utils.parseStrBlock(script_line)

        # アクションのみ場合
        if len(objs) == 0:
            result_list.append(script_line)
            continue
        elif indexs[0].isdigit() and indexs[0] != '1': ## Only for accurate ID indication
            if objs[0] in rooms:
                current_camera_room = objs[0]
            else:
                edges = graph['edges']
                rooms_object_exist = [edge['to_id'] for edge in edges if edge['relation_type'] == 'INSIDE' and edge['from_id'] == int(indexs[0])]
                room_name_object_exist = [node['class_name'] for node in graph['nodes'] if node['id'] in rooms_object_exist and node['class_name'] in rooms][0]
                current_camera_room = room_name_object_exist
            keyword = (objs[0], int(indexs[0]))
            if obj_id_map.get(keyword) is None:
                obj_id_map[keyword] = int(indexs[0])
            result_list.append(script_line)
            if action not in ['WALK', 'FIND', 'PUTOBJBACK'] and len(custom_cameras) > 0:
                custom_cameras.append(custom_cameras[len(custom_cameras)-1])
            else:
                custom_cameras.append(available_fixed_camera(current_camera_room))
            continue
        else:
            first_obj = objs[0]
            if len(objs) > 1:
                second_obj = objs[1]
            else:
                second_obj = None
            # ①graphに探す→②class_name_equivalenceに探す→③alternative_temp_objに探す
            first_obj_index = get_obj_index(first_obj, action)
            if first_obj_index == '1':
                script_line, first_obj_index = replace_obj_and_get_index(script_line, first_obj)
            if second_obj is not None and script_line is not None:
                second_obj_index = get_obj_index(second_obj, action)
                if second_obj_index == '1':
                    script_line, second_obj_index = replace_obj_and_get_index(script_line, second_obj)
            else:
                second_obj_index = None

            if script_line is not None:
                script_line = replace_index(script_line, first_obj_index, second_obj_index)
                action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
                keyword = (objs[0], int(indexs[0]))
                if obj_id_map.get(keyword) is None:
                    obj_id_map[keyword] = int(indexs[0])
                result_list.append(script_line)

        if action not in ['WALK', 'FIND', 'PUTOBJBACK'] and len(custom_cameras) > 0:
            custom_cameras.append(custom_cameras[len(custom_cameras)-1])
        else:
            custom_cameras.append(available_fixed_camera(current_camera_room))
    return result_list

def available_fixed_camera(room_name):
    mode_list = ["PERSON_FROM_BACK", "AUTO"]
    mode_map = fixed_cameras[room_name]
    return mode_list + mode_map[scene_id]

In [61]:
scene_id = 1
program_files = []
data_path = "../dataset/Test/scene"+str(scene_id)+"/*.txt"
for file_path in sorted(glob.glob(data_path)):
    file_name = file_path.replace(file_path[0:file_path.rfind("\\")+1], "")
    
    program_name, description, list_of_steps = get_program_file(file_path)
    program_files.append({
        "file_name":file_name,
        "name": program_name,
        "description": description,
        "list_of_steps": list_of_steps
    })

In [63]:
comm = UnityCommunication()
tsv_rows=[]

for program in program_files:
    try:
        print('\nStarting',program["file_name"])
        message = comm.reset(scene_id - 1)
        su, graph = comm.environment_graph()
        program_name = program["name"]
        script_lines = program["list_of_steps"]
        tsv_data={
            'file_name':program["file_name"],
            'script':[],
            'errors':[],
            'results':[],
            'duration':[],
        }

#         add_object_out_of_script('potato', 'kitchen', 'kitchentable', 'ON')

        current_room = 'livingroom' # Initial Room : Need to change according to activity　＊＊＊＊＊＊＊
        new_graph_state_list = []
        obj_id_map = {}
        custom_cameras = []
        current_camera_room = current_room
        script_lines = correct_script(script_lines)
        comm.add_character(initial_room=current_room)
        su, graph = comm.environment_graph()

        try:
            preconds = add_preconds.get_preconds_script(script_lines).printCondsJSON()
        except Exception as ex:
            tsv_data={
                'file_name':program["file_name"],
                'script':' '.join(script_lines),
                'errors':[ex],
                'results':[],
                'duration':[],
            }
            tsv_rows.append(tsv_data)
            continue
        graph = switchOff(graph)
        print('Preconds\n',preconds,'\n')

        modified_graph = check_programs.translate_graph_dict_nofile(graph)
        info = check_programs.check_script(script_lines, preconds, graph_path=None, inp_graph_dict=modified_graph)##, id_mapping=obj_id_map, modify_graph=False, info={'room_mapping': room_map}
        message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
        graph_state_list = delete2000(graph_state_list)
        print(message)

        if message != 'Script is executable':
            tsv_data['script'].append(' '.join(script_lines))
            tsv_data['errors'].append(message)
            tsv_rows.append(tsv_data)
            continue

        i=0
        executed_lines =[]
        for line in script_lines:
            line_to_execute = '<char0> '+line
            executed_lines.append(line_to_execute)
            success, g = comm.environment_graph()
            g = update_states(g, graph_state_list[i], obj_id_map)##, modified_graph=modified_graph if i == 0 else None
            new_graph_state_list.append(copy.deepcopy(g))

            start = time.time()
            s, message = comm.render_script(
                [line_to_execute],
                recording=True, 
                find_solution=False, 
                frame_rate=30,
                vis_check_object=False, 
                vis_check_character=False,
                camera_mode=custom_cameras[i],#"PERSON_TOP"
                file_name_prefix=program['name'],
                output_folder="Output/scene" + str(scene_id))
            print(str(s) + ": " + str(line_to_execute) + (str(" >>> " + str(message)) if s==False else ''))
            tsv_data['script'].append(line_to_execute)
            tsv_data['results'].append(message)
            tsv_data['duration'].append(time.time() - start)
            i+=1

        tsv_rows.append(tsv_data)
        export(new_graph_state_list, executed_lines, program['name'], program['description'], tsv_data['duration'])
    except Exception as e:
        print(e)
        tsv_data={
            'file_name':program["file_name"],
            'script':' '.join(program["list_of_steps"]),
            'errors':[e],
            'results':[],
            'duration':[],
        }
        tsv_rows.append(tsv_data)
        continue

with open('../simulation/Output/logs_scene'+str(scene_id)+'.tsv', 'wt', newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t', dialect="excel")
    tsv_writer.writerow(['ファイル名', 'スクリプト', 'エラー', '実行結果', '期間'])
    for row in tsv_rows:
        error_script = row['script'][:len(row['errors'])]
        for script, error in zip(error_script, row['errors']):
            tsv_writer.writerow([row['file_name'], script, error])
        if len(row['results']) > 0:
            result_script = row['script'][len(row['errors']):]
            for script, result, duration in zip(result_script, row['results'], row['duration']):
                tsv_writer.writerow([row['file_name'], script, '', result, duration])


Starting Clean television.txt
Preconds
 [{'location': [['tv', '430'], ['livingroom', '339']]}, {'location': [['towel', '55'], ['livingroom', '339']]}, {'grabbed': ['towel', '55']}] 

Script is executable
True: <char0> [WALK] <tv> (430)
True: <char0> [WALK] <towel> (55)
True: <char0> [GRAB] <towel> (55)
True: <char0> [WALK] <livingroom> (339)
True: <char0> [WALK] <tv> (430)
True: <char0> [WIPE] <towel> (55) <tv> (430)


In [64]:
#### complementing missing images
try:
    for x in range(6):
        for program in program_files:
            activity_name = program["file_name"]
            activity_name = activity_name[:-4]
            img_path = "../simulation/Output/scene"+str(scene_id) + "/" + activity_name + "/0/??????_????_" + str(x) + "_normal.png"
            i = 0
            for file_path in sorted(glob.glob(img_path)):
                file_name = file_path.replace("../simulation/Output/scene"+str(scene_id) + "/" + activity_name + "/0\\", "")
                file_num = file_name.split("_")[1]
                if i != int(file_num):
                    print(str(i) + " " + str(int(file_num)))
                    img = PIL.Image.open(file_path)
                    new_file_name = "../simulation/Output/scene"+str(scene_id) + "/" + activity_name + "/0/" + "Action_%04d_" + str(x) + "_normal.png"
                    img.save(new_file_name % i)
                    print(file_name)
                    print(file_name + " - 1")
                    print("save")
                    i += 1
                i += 1
except Exception as e:
    print(e)

In [ ]:
import subprocess

dir_path = "../simulation/Output/scene"+str(scene_id)+"/"
try:
    for x in range(6):
        for activity in program_files:
            file_name = activity["file_name"]
            activity_name = activity["name"]
            file_name = file_name[:-4]
            if os.path.exists(dir_path + file_name) == True:
                sh = subprocess.check_output(["ffmpeg", "-framerate", "14.5", "-i", dir_path + file_name + "/0/Action_%04d_" + str(x) + "_normal.png", "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-r", "30", dir_path + file_name + "/0/" + activity_name.replace(" ", "_") + str(x) + ".mp4"])
                print(sh)
except Exception as e:
    print(e)

In [ ]:
comm = UnityCommunication()
message = comm.reset(3)
su, graph = comm.environment_graph()
print(graph)